Define the geometry of the problem, we assume that the domain is a rectangle. 
Everything marked with a `[*]` can be modified by user needs.

In [1]:
from porepy.fracs import importer

#[*] path for the fracture network file name
fracture_file_name = 'network.csv'

#[*] folder to store the simulation results
folder_name = 'results'

#[*] activate to export solution in vtu format (for ParaView)
save_solution = True

#[*] define the mesh size of the problem
mesh_size = 400

# geometrical tolerance for the computation
tol = 1e-7

# pack the data for the grid generation
mesh_kwargs = {'tol': tol}
mesh_kwargs['mesh_size'] = {'mode': 'weighted', 'value': mesh_size, 'bound_value': mesh_size}

# import the fractures and create the grid
gb = importer.dfm_2d_from_csv(fracture_file_name, mesh_kwargs)
gb.compute_geometry()
gb.assign_node_ordering()

# define the computational domain (useful for boundary conditions)
# by default takes the bounding box of the fracture network
domain = gb.bounding_box(as_dict=True)

if save_solution:
    from porepy.viz import exporter

    save = exporter.Exporter(gb, 'grid', 'grid')
    save.write_vtk()

Gmsh processed file successfully


Grid creation completed. Elapsed time 0.4433135986328125


Created 1 2-d grids with 12530 cells
Created 63 1-d grids with 1317 cells
Created 85 0-d grids with 85 cells




Define the physical data for a single-phase incompressible flow model.

In [6]:
import numpy as np

#[*] define the fracture and rock matrix permeability, unit [m^2]
fracture_permeability = 1e-8
rock_matrix_permeability = 1e-12

# consistency check for permeability
assert fracture_permeability > 0 and rock_matrix_permeability > 0

#[*] define the fracture aperture, unit [m]
fracture_aperture = 1e-2

# consistency check for fracture aperture
assert fracture_aperture > 0

#[*] end time for the simulation [s]
end_time = 10000 * 1e7 * np.pi

#[*] number of time steps
number_time_steps = 10

#[*] define the pressure boundary conditions, unit [Pa] 
# use key 'left', 'right', 'top', 'bottom' to define which portion of the boundary
# NOTE: use 'left' and 'right' or 'top' and 'bottom'
dirichlet_boundary = {}
dirichlet_boundary['left'] = 0
dirichlet_boundary['right'] = 1013250

# consistency check for boundary condition
assert len(dirichlet_boundary) == 2 and \
       ('left' in dirichlet_boundary and 'right' in dirichlet_boundary) or \
       ('top' in dirichlet_boundary and 'bottom' in dirichlet_boundary)

# pack the problem data in a single dictionary
data_problem = {'domain': domain, 'tol': tol, 'dirichlet_boundary': dirichlet_boundary,
                'kf': fracture_permeability, 'km': rock_matrix_permeability,
                'aperture': fracture_aperture, 
                'end_time': end_time, 'number_time_steps': number_time_steps,
                'folder_name': folder_name}

Run single-phase flow model

In [7]:
from darcy_and_transport_data import Simulation

simulation = Simulation(gb, data_problem)
simulation.solve()
simulation.save()

Solve elliptic model
Discretize
Done. Elapsed time 5.959989309310913
Solve linear system using direct solver
Done. Elapsed time 0.09331369400024414
Time stepping using 10 steps
Step 1 out of 10
Step 2 out of 10
Step 3 out of 10
Step 4 out of 10
Step 5 out of 10
Step 6 out of 10
Step 7 out of 10
Step 8 out of 10
Step 9 out of 10
Step 10 out of 10


Define the data for a transport model, the flux field is given by the previous model

[{'bound_flux': <1796x1796 sparse matrix of type '<class 'numpy.float64'>'
  	with 232 stored elements in COOrdinate format>,
  'deltaT': 5.0,
  'discharge': array([ 0.        ,  0.05119577,  0.06078984, ...,  0.        ,
          0.        ,  0.        ]),
  'dof': 1120,
  'file_name': 'transport_000000.vtu',
  'flux': <1796x1120 sparse matrix of type '<class 'numpy.float64'>'
  	with 3360 stored elements in COOrdinate format>,
  'grid_dim': array([2, 2, 2, ..., 2, 2, 2]),
  'node_number': 0,
  'param': Data object for grid with 1120 cells and 1796 faces 
  Assigned attributes / properties: 
  ['_num_cells', '_num_faces', 'dim', 'g', 'known_physics', '_perm', '_bc_flow', '_bc_val_flow', '_source_flow', '_apertures', '_conductivity', '_bc_transport', '_bc_val_transport', '_source_transport'],
  'pressure': array([ 0.19124628,  0.29150889,  0.72772383, ...,  0.21809105,
          0.2489542 ,  0.47300344]),
  'problem': <darcy_and_transport_data.FlowData at 0x7fcaad44f7b8>,
  'transport

Run transport flow-model

Compute production